In [13]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
import json


In [5]:
provincias = pd.read_csv('raw_data/provincias.csv')
energy = pd.read_csv('raw_data/Spain_energy_df_2017.csv')



In [40]:
with urlopen('https://raw.githubusercontent.com/deldersveld/topojson/master/countries/spain/spain-province.json')\
    as response:
    Spain = json.load(response)

In [43]:
provincias_grouped = provincias.groupby('Provincia').agg({'latitude':'mean','longitude':'mean'}).reset_index()

In [65]:
energy.head(2)

,Unnamed: 0,country,capacity_mw,lat,lon,primary_fuel,Solar Generation (2017)
0,20135,ESP,877.66,43.5528,-5.7231,Coal,4003.64
1,20136,ESP,758.74,39.9427,-3.8548,Gas,1798.46


In [79]:
def get_closest_province(row):
    df = provincias_grouped[['latitude','longitude']] - list(row[['lat','lon']])

    df['agg'] = df['latitude']**2 + df['longitude']**2
    name_index = df.sort_values('agg').iloc[0,:].name

    return provincias_grouped.iloc[name_index].Provincia
energy['province'] = energy.apply(get_closest_province,axis=1)

energy.groupby('province').agg({'Solar Generation (2017)':'sum'}).sort_values('Solar Generation (2017)',ascending=False).head(3)

,Solar Generation (2017)
province,
Tarragona,29594.68
Caceres,18406.04
Valencia/Val�ncia,15133.59


In [81]:
energy.sample(3)

,Unnamed: 0,country,capacity_mw,lat,lon,primary_fuel,Solar Generation (2017),province
50,20185,ESP,9.6,37.8370,-4.9800,Solar,19.24,Cordoba
112,20247,ESP,49.9,37.7574,-5.0620,Solar,102.69,Cordoba
587,20722,ESP,14.4,38.4000,-6.4667,Wind,30.62,Badajoz
